In [1]:
%matplotlib inline

In [2]:
import pandas
import numpy
import root_numpy
from hep_ml.nnet import MLPMultiClassifier
from rep.metaml import FoldingClassifier
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score

## Reading features

In [3]:
import json
with open('features.json') as f:
    feature_families = json.load(f)

In [4]:
train_features = numpy.concatenate(feature_families.values())
print len(train_features)
print train_features

60
[u'MuonNShared' u'MuonIsLooseMuon' u'MuonIsMuon' u'MuonBkgLL' u'MuonMuLL'
 u'TrackFitVeloChi2' u'TrackFitVeloNDoF' u'TrackFitMatchChi2'
 u'TrackGhostProbability' u'TrackP' u'TrackChi2PerDof' u'TrackFitTChi2'
 u'TrackPt' u'TrackNumDof' u'TrackFitTNDoF' u'TrackDOCA' u'InAccSpd'
 u'InAccPrs' u'InAccBrem' u'InAccEcal' u'InAccHcal' u'InAccMuon'
 u'CombDLLmu' u'CombDLLpi' u'CombDLLp' u'CombDLLe' u'CombDLLk'
 u'RichAboveMuThres' u'RichAboveElThres' u'RichAbovePiThres'
 u'RichAboveKaThres' u'RichAbovePrThres' u'RichUsedR1Gas' u'RichUsedR2Gas'
 u'RichDLLbt' u'RichDLLpi' u'RichDLLe' u'RichDLLp' u'RichDLLmu' u'RichDLLk'
 u'CaloBremMatch' u'CaloElectronMatch' u'CaloTrMatch' u'CaloTrajectoryL'
 u'CaloChargedSpd' u'CaloChargedPrs' u'CaloChargedEcal' u'CaloNeutralSpd'
 u'CaloNeutralPrs' u'CaloNeutralEcal' u'CaloSpdE' u'CaloPrsE' u'CaloEcalE'
 u'CaloHcalE' u'EcalPIDmu' u'HcalPIDmu' u'PrsPIDe' u'BremPIDe' u'EcalPIDe'
 u'HcalPIDe']


## Read the data

In [5]:
data_train = pandas.DataFrame(root_numpy.root2array('../data/global_train.root', 'tree', step=10))
data_test  = pandas.DataFrame(root_numpy.root2array('../data/global_test.root', 'tree', step=10))

from utils import shrink_floats
shrink_floats(data_train)
shrink_floats(data_test)

In [6]:
from collections import Counter
Counter(data_train.MCParticleType)

Counter({-2212: 47733,
         -321: 49353,
         -211: 50040,
         -13: 50197,
         -11: 50361,
         0: 100002,
         11: 49625,
         13: 49813,
         211: 49974,
         321: 50632,
         2212: 52270})

In [7]:
data_train.head()

,VeloCharge,BremPIDe,CaloNeutralPrs,CaloNeutralSpd,InAccBrem,InAccSpd,CaloPrsE,InAccPrs,HcalPIDe,CaloHcalE,...,piplus_OWNPV_XERR,piplus_OWNPV_YERR,piplus_OWNPV_ZERR,piplus_OWNPV_CHI2,piplus_OWNPV_NDOF,piplus_IP_OWNPV,piplus_IPCHI2_OWNPV,nCandidate,totCandidates,EventInSequence
0,1.028090,-999.000000,-999.000000,-999,0,0,-999.000000,0,-999.000000,-999.000000,...,0.0096,0.0096,0.0532,33.816654,83,0.111340,15.622943,16,37,9099
1,1.189990,-0.618836,35.381054,0,1,1,122.873741,1,0.961872,0.000000,...,0.0089,0.0085,0.0427,53.822292,123,0.802176,196.415192,2,71,3135
2,1.183287,-0.082057,13.987859,0,1,0,-999.000000,0,-999.000000,-999.000000,...,0.0097,0.0097,0.0559,34.384235,85,0.061936,1.096196,20,104,10176
3,1.026841,2.711674,9.873783,0,1,1,50.466003,1,0.434916,0.000000,...,0.0131,0.0127,0.0683,22.266401,49,0.055527,2.439735,7,84,205
4,1.158708,-999.000000,-999.000000,-999,0,1,36.203873,1,-1.160355,3640.665527,...,0.0083,0.0083,0.0567,52.799332,123,0.176878,1.941835,57,107,13362


#### add signal column (from 0 to 5 classes), weights (to balance data)

In [8]:
from utils import compute_labels_and_weights, compute_charges, names_labels_correspondence, labels_names_correspondence
from utils import plot_hist_features, roc_auc_score_one_vs_all, compute_cum_sum, convert_DLL_to_LL

In [9]:
data_train['Signal'], data_train['Weight'] = compute_labels_and_weights(data_train.MCParticleType.values)
data_test['Signal'], data_test['Weight'] = compute_labels_and_weights(data_test.MCParticleType.values)

In [10]:
for family_name, family_features in feature_families.items():
    print "{:10}".format(family_name), len(family_features), '\t', family_features

muon       5 	[u'MuonNShared', u'MuonIsLooseMuon', u'MuonIsMuon', u'MuonBkgLL', u'MuonMuLL']
track      11 	[u'TrackFitVeloChi2', u'TrackFitVeloNDoF', u'TrackFitMatchChi2', u'TrackGhostProbability', u'TrackP', u'TrackChi2PerDof', u'TrackFitTChi2', u'TrackPt', u'TrackNumDof', u'TrackFitTNDoF', u'TrackDOCA']
acceptance 6 	[u'InAccSpd', u'InAccPrs', u'InAccBrem', u'InAccEcal', u'InAccHcal', u'InAccMuon']
DLL        5 	[u'CombDLLmu', u'CombDLLpi', u'CombDLLp', u'CombDLLe', u'CombDLLk']
RICH       13 	[u'RichAboveMuThres', u'RichAboveElThres', u'RichAbovePiThres', u'RichAboveKaThres', u'RichAbovePrThres', u'RichUsedR1Gas', u'RichUsedR2Gas', u'RichDLLbt', u'RichDLLpi', u'RichDLLe', u'RichDLLp', u'RichDLLmu', u'RichDLLk']
CALO       20 	[u'CaloBremMatch', u'CaloElectronMatch', u'CaloTrMatch', u'CaloTrajectoryL', u'CaloChargedSpd', u'CaloChargedPrs', u'CaloChargedEcal', u'CaloNeutralSpd', u'CaloNeutralPrs', u'CaloNeutralEcal', u'CaloSpdE', u'CaloPrsE', u'CaloEcalE', u'CaloHcalE', u'EcalPIDmu',

## Function to print quality

In [12]:
from utils import roc_auc_score_one_vs_all

In [14]:
%%time
mlp_clf = SklearnClassifier(MLPMultiClassifier(layers=[20], epochs=100, scaler='iron'), 
                            features=train_features)
mlp_clf.fit(data_train, data_train.Signal)

CPU times: user 2min 5s, sys: 2.64 s, total: 2min 8s
Wall time: 1min 54s


## Checking simple MLP

In [ ]:
%%time
mlp_clf = SklearnClassifier(MLPMultiClassifier(layers=[40, 30, 20], epochs=1000, scaler='iron'), 
                            features=train_features)
mlp_clf.fit(data_train, data_train.Signal)

CPU times: user 1h 52min 42s, sys: 1h 19min 32s, total: 3h 12min 14s
Wall time: 31min 13s


In [13]:
from rep.estimators import SklearnClassifier
from hep_ml.nnet import MLPMultiClassifier

In [ ]:
%%time
mlp_clf = SklearnClassifier(MLPMultiClassifier(layers=[40, 30, 20], epochs=1000, scaler='iron'), 
                            features=train_features)
mlp_clf.fit(data_train, data_train.Signal)

CPU times: user 1h 52min 42s, sys: 1h 19min 32s, total: 3h 12min 14s
Wall time: 31min 13s


In [26]:
print_aucs(mlp_clf, data_test)

Ghost    0.9579   Electron 0.9889   Muon     0.9908   Pion     0.9538   Kaon     0.9254   Proton   0.9252   


In [27]:
print_aucs(mlp_clf, data_train)

Ghost    0.9583   Electron 0.9893   Muon     0.9912   Pion     0.9544   Kaon     0.9259   Proton   0.9257   


## Defining one more network over hep_ml.nnet

In [13]:
from hep_ml.nnet import MLPMultiClassifier, MLPBase

In [14]:
import theano.tensor as T
class FamiliesMultiClassifier(MLPMultiClassifier):
    """
    Has two layers = one over each subsystem, and one is global.
    """
    def prepare(self):
        print self.layers_
        n_input, subsystem_sizes, size2, size3, n_output = self.layers_
        
        subsystem_offsets = numpy.cumsum([0] + list(subsystem_sizes))
        assert n_input == sum(subsystem_sizes) + 1
        W1s = []
        subsystem_output = 10
        for i, subsystem_size in enumerate(subsystem_sizes):
            W1s.append(self._create_matrix_parameter('W1_{}'.format(i), subsystem_size + 1, subsystem_output))
        W2 = self._create_matrix_parameter('W2', subsystem_output * len(subsystem_sizes), size2)
        W3 = self._create_matrix_parameter('W3', size2, size3)
        W4 = self._create_matrix_parameter('W4', size3, n_output)

        def my_activation(x):
            outputs = []
            for i, W1_ in enumerate(W1s):
                features_range = list(numpy.arange(subsystem_offsets[i], subsystem_offsets[i+1])) + [n_input - 1]
                # print 'features_range', features_range
                output = T.tanh(T.dot(x[:, features_range], W1_))
                outputs.append(output)

            layer1 = T.concatenate(outputs, axis=1)
            layer2 = T.tanh(T.dot(layer1, W2))
            layer3 = T.tanh(T.dot(layer2, W3))
            layer4 = T.dot(layer3, W4)
            return layer4    
                

        return my_activation

In [60]:
%%time
layers = [[len(x) for x in feature_families.values()], 30, 20]
fmc_clf = SklearnClassifier(FamiliesMultiClassifier(layers=layers, epochs=1000, scaler='iron'), 
                            features=train_features)
fmc_clf.fit(data_train, data_train.Signal)

[61, [5, 11, 6, 5, 13, 20], 30, 20, 6]
CPU times: user 3h 35min 45s, sys: 1h 53min 22s, total: 5h 29min 8s
Wall time: 2h 59min 16s


In [20]:
import cPickle
# with open('./structured_NN.pkl', 'w') as f:
#     cPickle.dump(fmc_clf, f)

with open('./structured_NN.pkl') as f:
    fmc_clf = cPickle.load(f)

In [21]:
def decision_function(self, X):
    """
    Activates NN on particular dataset

    :param numpy.array X: of shape [n_samples, n_features]
    :return: numpy.array with results of shape [n_samples]
    """
    X = self._transform(X, fit=False).astype('float32')
    return self.Activation(X)
    
FamiliesMultiClassifier.decision_function = decision_function

In [22]:
print_aucs(fmc_clf, data_test)

Ghost    0.9575   Electron 0.9885   Muon     0.9905   Pion     0.9529   Kaon     0.9244   Proton   0.9242   


In [24]:
print_aucs(fmc_clf, data_train)

Ghost    0.9578   Electron 0.9889   Muon     0.9908   Pion     0.9533   Kaon     0.9247   Proton   0.9246   


## Keras

### Preprocessing

In [21]:
from hep_ml.preprocessing import IronTransformer

scaler = IronTransformer(symmetrize=True)
scaler.fit(data_train[train_features])
new_trainX = scaler.transform(data_train[train_features])
trainY = data_train.Signal.values

In [172]:
from sklearn.preprocessing import StandardScaler
st_scaler = StandardScaler()
st_scaler.fit(data_train[train_features])

StandardScaler(copy=True, with_mean=True, with_std=True)

In [22]:
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import BatchNormalization
from keras.models import Sequential
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

## Single layer with StandardScaler

In [178]:
model = Sequential()
model.add(Dense(150, input_dim=len(train_features)))
model.add(Activation('tanh'))

n_classes = len(set(trainY))
model.add(Dense(n_classes))
model.add(Activation('softmax'))

keras_no_iron_clf = model
keras_no_iron_clf.compile(loss='categorical_crossentropy', optimizer=Adam())

In [179]:
%%time
keras_no_iron_predictions = []


for epochs in range(15):
    keras_no_iron_clf.fit(st_scaler.transform(data_train[train_features]), np_utils.to_categorical(trainY), 
                          verbose=0, nb_epoch=5, batch_size=256)
    print 'done'
    keras_no_iron_predictions.append(keras_no_iron_clf.predict_proba(st_scaler.transform(data_test[train_features]), verbose=False))

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
CPU times: user 23min 49s, sys: 1h 6min 14s, total: 1h 30min 3s
Wall time: 13min 8s


In [181]:
for p in keras_no_iron_predictions:
    print roc_auc_score_one_vs_all(data_test.Signal, p, data_test.Weight).values

[[ 0.95233476  0.98731826  0.98792387  0.94891918  0.92129942  0.92129873]]
[[ 0.95500032  0.98795841  0.98895797  0.95065482  0.9244877   0.92369024]]
[[ 0.95611325  0.98817854  0.98942338  0.95183913  0.92432307  0.92495718]]
[[ 0.9565099   0.98831205  0.9896617   0.95231596  0.92557563  0.92567358]]
[[ 0.95704975  0.98869146  0.98993983  0.95198752  0.92566623  0.92555023]]
[[ 0.95800668  0.98871423  0.99003609  0.9525334   0.92488414  0.924423  ]]
[[ 0.95753114  0.98845578  0.99005091  0.95306655  0.92697531  0.92676571]]
[[ 0.95824846  0.98887089  0.99015913  0.95308091  0.92618218  0.92606379]]
[[ 0.958308    0.98862266  0.99009165  0.95297046  0.9269928   0.92668938]]
[[ 0.95812997  0.98850111  0.99013517  0.95280935  0.92654768  0.92696559]]
[[ 0.9580592   0.98893064  0.99014577  0.95322719  0.92685258  0.9271297 ]]
[[ 0.95856254  0.98911537  0.99026655  0.95354437  0.92668991  0.92684252]]
[[ 0.95736746  0.98886616  0.99040561  0.95342798  0.92700319  0.92688933]]
[[ 0.9585797

## Single layer

In [159]:
model = Sequential()
model.add(Dense(150, input_dim=len(train_features)))
model.add(Activation('tanh'))

n_classes = len(set(trainY))
model.add(Dense(n_classes))
model.add(Activation('softmax'))

keras_onelayer_clf = model
keras_onelayer_clf.compile(loss='categorical_crossentropy', optimizer=Adam())

In [164]:
%%time
# keras_onelayer_predictions = []
for epochs in range(15):
    keras_onelayer_clf.fit(new_trainX.values, np_utils.to_categorical(trainY), 
                           verbose=0, nb_epoch=5, batch_size=256)
    print 'done'
    keras_onelayer_predictions.append(keras_onelayer_clf.predict_proba(scaler.transform(data_test[train_features]).values, verbose=False))

done
done
done
done
done
CPU times: user 6min 17s, sys: 15min 58s, total: 22min 16s
Wall time: 3min 23s


In [166]:
for p in keras_onelayer_predictions:
    print roc_auc_score_one_vs_all(data_test.Signal, p, data_test.Weight).values

[[ 0.95666376  0.98789876  0.99014538  0.95192405  0.92033015  0.91941495]]
[[ 0.95825367  0.9886869   0.99079318  0.95346667  0.92338571  0.92270784]]
[[ 0.95856513  0.98895444  0.99103536  0.95387046  0.92436585  0.92385327]]
[[ 0.95869598  0.98915723  0.99116502  0.95457781  0.9248634   0.92391089]]
[[ 0.95890223  0.98922488  0.99124332  0.9547234   0.924765    0.9239011 ]]
[[ 0.95869938  0.98940489  0.99131735  0.95493054  0.92508832  0.92518813]]
[[ 0.95887798  0.98937772  0.99140583  0.955043    0.92521522  0.92502914]]
[[ 0.95881337  0.98935549  0.99134941  0.95493522  0.92499898  0.92494409]]
[[ 0.95880484  0.9894924   0.99136138  0.95512026  0.92552391  0.92491172]]
[[ 0.95878197  0.98964464  0.99153951  0.95507195  0.92554708  0.92586741]]
[[ 0.95905589  0.98973771  0.99154188  0.95508617  0.92531566  0.9256772 ]]
[[ 0.95899474  0.98973991  0.99152305  0.95497449  0.92575512  0.92546164]]
[[ 0.95875155  0.98972273  0.99155035  0.95507517  0.92550633  0.92568639]]
[[ 0.9589660

In [171]:
save_predictions(keras_onelayer_clf.predict_proba(scaler.transform(data_test_full[train_features]).values, verbose=0), 
                 filename='./models/keras_onelayer_probs.pkl')

## Deep learning

In [182]:
%%time

model = Sequential()
model.add(Dense(300, input_dim=len(train_features)))
model.add(Dropout(0.1))
model.add(Activation('relu'))

model.add(Dense(300))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Activation('relu'))

model.add(Dense(400))
model.add(Dropout(0.5))
model.add(Activation('relu'))

n_classes = len(set(trainY))

model.add(Dense(n_classes))
model.add(Activation('softmax'))

keras_dl_clf = model
keras_dl_clf.compile(loss='categorical_crossentropy', optimizer=Adam())

CPU times: user 420 ms, sys: 0 ns, total: 420 ms
Wall time: 418 ms


In [183]:
keras_dl_predictions = []

In [187]:
%%time
for epochs in range(5):
    keras_dl_clf.fit(new_trainX.values, np_utils.to_categorical(trainY), 
                     verbose=0, nb_epoch=10, batch_size=256)
    print 'done'
    keras_dl_predictions.append(keras_dl_clf.predict_proba(scaler.transform(data_test[train_features]).values, verbose=False))

done


INFO (theano.gof.compilelock): Refreshing lock /home/robot-cern-ipcluster/.theano/compiledir_Linux-3.2--virtual-x86_64-with-debian-wheezy-sid-x86_64-2.7.11-64/lock_dir/lock


done
done
done
done
CPU times: user 2h 12min 14s, sys: 3h 58min 21s, total: 6h 10min 35s
Wall time: 53min 14s


In [188]:
for p in keras_dl_predictions:
    print roc_auc_score_one_vs_all(data_test.Signal, p, data_test.Weight).values

[[ 0.95974343  0.98996496  0.99166346  0.95572337  0.92788679  0.92757497]]
[[ 0.96042376  0.99017225  0.99178153  0.9561138   0.92778428  0.92742485]]
[[ 0.96053858  0.99026935  0.99192785  0.95622811  0.92860987  0.92815673]]
[[ 0.96039781  0.9903791   0.9917358   0.95620892  0.92882523  0.92873579]]
[[ 0.96018208  0.9902056   0.99181709  0.95599073  0.92882642  0.92868044]]


In [191]:
save_predictions(keras_dl_clf.predict_proba(scaler.transform(data_test_full[train_features]).values, verbose=False),
                 filename='./models/keras_dl_probs.pkl')

## Keras with sublayers

In [133]:
from keras.layers import Layer
from keras import backend as K

class Sublayer(Layer):
    def __init__(self, sizes, output_dim, **kwargs):
        self.output_dim = output_dim
        self.sizes = sizes
        super(Sublayer, self).__init__(**kwargs)

    def build(self, input_shape):
        input_dim = input_shape[1]
        self.trainable_weights = []
        for i in range(len(self.sizes) - 1):
            matrix_size = (self.sizes[i+1] - self.sizes[i], self.output_dim)
            initial_weight_value = numpy.random.random(size=matrix_size)
            W = K.variable(initial_weight_value)
            self.trainable_weights.append(W)
            print matrix_size
        self.trainable_weights.append(K.zeros((self.output_dim * (len(self.sizes) - 1),)))

    def call(self, x, mask=None):
        sublayers_outputs = []
        assert len(self.trainable_weights) == (len(self.sizes) - 1) + 1
        for i in range(len(self.sizes) - 1):
            w = self.trainable_weights[i]
            sublayers_outputs.append(K.dot(x[:, self.sizes[i]:self.sizes[i+1]], w))
   
        return K.concatenate(sublayers_outputs, axis=1) + self.trainable_weights[-1]

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], self.output_dim * (len(self.sizes) - 1))

In [131]:
from theano import tensor
from keras.layers import InputLayer

In [138]:
# %%time

model = Sequential()
model.add(Activation('linear', input_shape=[len(train_features)]))

sublayer_sizes = numpy.cumsum([0] + map(len, feature_families.values()))
print sublayer_sizes
model.add(Sublayer(sizes=sublayer_sizes, output_dim=50))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(300))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Activation('relu'))

model.add(Dense(400))
model.add(Dropout(0.5))
model.add(Activation('relu'))

n_classes = len(set(trainY))

model.add(Dense(n_classes))
model.add(Activation('softmax'))

keras_clf = model
keras_clf.compile(loss='categorical_crossentropy', optimizer=Adam())

[ 0  5 16 22 27 40 60]
(5, 50)
(11, 50)
(6, 50)
(5, 50)
(13, 50)
(20, 50)


/moosefs/miniconda/envs/ipython_py2/lib/python2.7/site-packages/keras/engine/topology.py:1537: UserWarning: Model inputs must come from a Keras Input layer, they cannot be the output of a previous non-Input layer. Here, a tensor specified as input to "sequential_38_model" was not an Input tensor, it was generated by layer activation_67.
Note that input tensors are instantiated via `tensor = Input(shape)`.
The tensor that caused the issue was: activation_input_19
  str(x.name))


In [ ]:
%%time 
keras_predictions = []
for epochs in range(10):
    model.fit(new_trainX.values, np_utils.to_categorical(trainY), verbose=0, nb_epoch=10, batch_size=256)
    print 'done'
    keras_predictions.append(keras_clf.predict_proba(scaler.transform(data_test[train_features]).values, verbose=False))

In [142]:
for p in keras_predictions:
    print roc_auc_score_one_vs_all(data_test.Signal, p, data_test.Weight).values

[[ 0.95964446  0.98971659  0.99164872  0.95547135  0.92751891  0.92765897]]
[[ 0.9603116   0.99011257  0.9919267   0.95603929  0.92908952  0.92911864]]
[[ 0.96005068  0.99034869  0.99200285  0.95633263  0.92905454  0.92911523]]
[[ 0.96066451  0.99034355  0.99216775  0.95652334  0.92893074  0.92861009]]
[[ 0.96060343  0.99034118  0.99216024  0.95665133  0.92935868  0.92960834]]
[[ 0.96077933  0.99041443  0.99211987  0.95658752  0.92937758  0.92896074]]
[[ 0.9609456   0.99043873  0.99217464  0.95680104  0.92934219  0.92931343]]
[[ 0.96097054  0.99052944  0.99211185  0.95658635  0.92962384  0.92949816]]
[[ 0.96102828  0.99051049  0.99222165  0.9568887   0.92923975  0.92946643]]
[[ 0.96102772  0.99047985  0.992204    0.95684819  0.92945688  0.92970952]]


In [106]:
for p in keras_predictions:
    print roc_auc_score_one_vs_all(data_test.Signal, p, data_test.Weight).values

[[ 0.95775714  0.98907129  0.99090353  0.95334663  0.92528597  0.92551847]]
[[ 0.95838697  0.98871884  0.99139109  0.95328123  0.92619441  0.92669809]]
[[ 0.95789139  0.98923812  0.99132039  0.95331727  0.92627813  0.92604281]]
[[ 0.95790181  0.9889717   0.99141375  0.95357187  0.92620091  0.92653914]]
[[ 0.9582516   0.98949188  0.99160778  0.95323862  0.92664485  0.92682461]]
[[ 0.95855145  0.98919276  0.99143765  0.95316899  0.9263546   0.92672128]]
[[ 0.95795853  0.9891986   0.99124588  0.95266372  0.92678905  0.92695552]]
[[ 0.95783303  0.98927905  0.99122803  0.95309603  0.92581961  0.92636813]]
[[ 0.95784982  0.98906583  0.99137406  0.95246268  0.92587585  0.92604555]]
[[ 0.95757006  0.98913391  0.99114323  0.95295626  0.92545959  0.92614058]]


In [143]:
data_test_full  = pandas.DataFrame(root_numpy.root2array('../data/global_test.root', 'tree', step=1))

In [144]:
predictions = keras_clf.predict_proba(scaler.transform(data_test_full[train_features]).values, verbose=False)

In [145]:
predictions.shape

(5999990, 6)

In [169]:
def save_predictions(predictions, filename):
    import cPickle
    saved_predictions = {
        i: predictions[:, i] for i in range(6)
    }
    with open(filename, 'w') as f:
        cPickle.dump(saved_predictions, f)

In [153]:
import cPickle

with open('models/keras_sublayers_probs.pkl', 'w') as f:
    cPickle.dump(saved_predictions, f)

## Keras for staged training

In [210]:
def return_model(input_dim, hidden_layer):
    model = Sequential()
    model.add(Dense(hidden_layer, input_dim=input_dim))
    # added dropout not to care about folding :)
    model.add(Dropout(p=0.3))
    model.add(Activation('tanh'))

    n_classes = len(set(trainY))
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))

    keras_onelayer_clf = model
    keras_onelayer_clf.compile(loss='categorical_crossentropy', optimizer=Adam())
    return keras_onelayer_clf

In [212]:
%%time
family_models = {}
for family_name, family_features in feature_families.items():
    family_model = return_model(len(family_features), hidden_layer=50)
    family_models[family_name] = family_model
    family_model.fit(new_trainX[family_features].values, np_utils.to_categorical(trainY), 
                     verbose=0, nb_epoch=20, batch_size=256)
    print 'done'
    

done
done
done
done
done
done
CPU times: user 8min 28s, sys: 4.25 s, total: 8min 32s
Wall time: 8min 32s


In [213]:
for family_name, family_features in feature_families.items():
    family_model = family_models[family_name]
    test_pred = family_model.predict_proba(scaler.transform(data_test[train_features])[family_features].values, verbose=False)
    print roc_auc_score_one_vs_all(data_test.Signal, test_pred, data_test.Weight).values

INFO (theano.gof.compilelock): Refreshing lock /home/robot-cern-ipcluster/.theano/compiledir_Linux-3.2--virtual-x86_64-with-debian-wheezy-sid-x86_64-2.7.11-64/lock_dir/lock


[[ 0.58331568  0.58943212  0.90738164  0.57859622  0.58123283  0.58869806]]
[[ 0.94679791  0.66227856  0.7793344   0.70914952  0.68253539  0.68137853]]
[[ 0.64807595  0.54112762  0.5890995   0.52057345  0.55321033  0.55598696]]
[[ 0.75520392  0.97438949  0.95504411  0.91173726  0.89168561  0.88275535]]
[[ 0.78141717  0.91491749  0.89614387  0.88260449  0.88270334  0.87715549]]
[[ 0.76730805  0.95250696  0.91187955  0.70156456  0.72602407  0.72978713]]


In [258]:
joint_train = [scaler.transform(data_train[train_features])]
joint_test  = [scaler.transform(data_test_full[train_features])]
for family_name, family_features in feature_families.items():
    family_model = family_models[family_name]
    train_pred = family_model.predict_proba(scaler.transform(data_train[train_features])[family_features].values, verbose=False)
    test_pred  = family_model.predict_proba(scaler.transform(data_test_full[train_features])[family_features].values, verbose=False)
    joint_train.append(pandas.DataFrame(train_pred))
    joint_test.append(pandas.DataFrame(test_pred))

In [259]:
train_full = pandas.concat(joint_train, axis=1)
test_full  = pandas.concat(joint_test, axis=1)

In [223]:
model = Sequential()

model.add(Dense(300, input_dim=train_full.shape[1]))
model.add(Dropout(0.3))
model.add(Activation('relu'))

model.add(Dense(400))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Activation('relu'))

n_classes = len(set(trainY))

model.add(Dense(n_classes))
model.add(Activation('softmax'))

keras_combiner_clf = model
keras_combiner_clf.compile(loss='categorical_crossentropy', optimizer=Adam())

In [ ]:
%%time 
# keras_combiner_predictions = []
for epochs in range(3):
    keras_combiner_clf.fit(train_full.values, np_utils.to_categorical(trainY), verbose=0, nb_epoch=5, batch_size=256)
    print 'done'
    keras_combiner_predictions.append(keras_combiner_clf.predict_proba(test_full.values, verbose=False))

In [262]:
for p in keras_combiner_predictions:
    print roc_auc_score_one_vs_all(data_test_full.Signal, p, data_test_full.Weight).values

[[ 0.95986957  0.98978866  0.99152842  0.95515659  0.92808501  0.92790364]]


In [265]:
save_predictions(keras_combiner_predictions[0], './models/keras_subdetectors_probs.pkl')